In [1]:
from youtube_api import YouTubeDataAPI
import pandas as pd
from apikey import api_key
from apikey import username
from apikey import password
import requests as rq
import re
import isodate
from sqlalchemy import create_engine

In [2]:
# creating the connection to upload the table to sql database

driver = 'mysql+pymysql'
ip = 'barcelona-db.cyxhqbnhiohl.eu-west-3.rds.amazonaws.com'
db = 'project3'
connection_string  = f'{driver}://{username}:{password}@{ip}/{db}'

In [3]:
engine = create_engine(connection_string)

In [4]:
# getting the channels dataframe from our database.
query = 'SELECT Channel FROM content_creator'

In [5]:
channel_df = pd.read_sql(query, engine)

In [6]:
# Converting a dataframe to a list to loop over it later.
channels = list(channel_df['Channel'])

In [9]:
channels

['elrubiusOMG',
 'VEGETTA777',
 'WillyRex',
 'Doggy Doggy Cartoons',
 'AuronPlay',
 'iTownGamePlay',
 'Makiman131',
 'Las Ratitas',
 'ExpCaseros',
 'Mejores Juguetes',
 'Toys And Funny Kids Play Doh Cartoons',
 'InfantiLandia',
 'TheGrefg',
 'DaniRep',
 'Luzugames ',
 'DalasReview',
 'Mikecrack',
 'Adexe &amp; Nau',
 'El Rincón De Giorgio',
 'ZarcortGame',
 'Wismichu',
 'Funny Stop Motion videos',
 'Ya – Alisa',
 'Alexby11',
 'sTaXxCraft',
 'FC Barcelona',
 'gymvirtual',
 'Pocoyo',
 'TOYS on the go!',
 'MangelRogel',
 'ElChurches',
 'TheShooterCoC',
 'StaxX',
 'patty dragona',
 'byViruZz',
 'DjMaRiiO',
 'LOLiTO FDEZ',
 'Experimentar En Casa',
 'Tri-line',
 'ElTrollino',
 'Keyblade',
 'Manucraft',
 'bytarifa',
 'Kronno Zomber',
 'Antena 3',
 'Alvaro845',
 'Real Madrid']

In [10]:
# Creating a Top3 list to through it
first3 = [channels[0], channels[1], channels[2]]

In [12]:
first3

['elrubiusOMG', 'VEGETTA777', 'WillyRex']

In [13]:
# Creating the access to the Youtube API
yt = YouTubeDataAPI(api_key)

In [14]:
# Defining a function to get videos given the name of a channel, limiting at 50.

def getVideos(channel):
    return yt.search(channel, max_results=50)

In [15]:
channelVideos = [] # this will be the list where we will append all the data got from the API

In [16]:
# we loop over each channel in 'channels' list to get the videos using the function.
for channel in first3:
    try:
        channelVideos.append(getVideos(channel))
    except:
        pass

In [17]:
len(channelVideos)

3

In [18]:
channelVideos

[[{'video_id': 'NgGqA9Sy0tA',
   'channel_title': 'elrubiusOMG',
   'channel_id': 'UCXazgXDIYyWH-yXLAkcrFxw',
   'video_publish_date': datetime.datetime(2019, 6, 20, 18, 16, 44),
   'video_title': 'WHISPER CHALLENGE con Auronplay',
   'video_description': 'Like por un puñetazo en la espalda Video de Auron https://www.youtube.com/watch?v=sboXdNY4g9I Twitch: https://twitch.tv/Rubius Server Discord: ...',
   'video_category': None,
   'video_thumbnail': 'https://i.ytimg.com/vi/NgGqA9Sy0tA/hqdefault.jpg',
   'collection_date': datetime.datetime(2019, 11, 13, 16, 58, 49, 103007)},
  {'video_id': 'jUyxgUkNo9U',
   'channel_title': 'elrubiusOMG',
   'channel_id': 'UCXazgXDIYyWH-yXLAkcrFxw',
   'video_publish_date': datetime.datetime(2019, 7, 13, 19, 59, 46),
   'video_title': 'AGUA DE CHICA GAMER 🤢',
   'video_description': 'Like si quieres agua de Mateo Twitch: https://twitch.tv/Rubius Server Discord: https://discord.gg/rubius Twittah: https://twitter.com/Rubiu5 FB: ...',
   'video_category'

In [19]:
# Retrieving a list of video_ids
channelName = []
videoIds = []
videoPublishDate = []
for i in range(len(channelVideos)):
    for video in channelVideos[i]:
        channelName.append(video.get('channel_title'))
        videoIds.append(video.get('video_id'))
        videoPublishDate.append(video.get('video_publish_date').strftime("%m/%d/%Y"))

In [38]:
Top3_df = pd.DataFrame({'Channel': channelName, 'Video_Id': videoIds, 'Published':videoPublishDate})

In [42]:
# Creating a csv file to don't need to query again to the API
Top3_df.to_csv('./data/Top3_dirty.csv', sep='\t', index = False)

In [43]:
Top3_df['Channel'].unique()

array(['elrubiusOMG', 'εlrubiusOMG', 'VEGETTA777', 'TheWillyrex',
       'Willyrex', 'Red Royale', 'Averroes', 'D3stri', 'atack3000',
       'Core Tops'], dtype=object)

In [40]:
# Checking how many rows from 'other' channels we have. If there are few, we drop them. 
# Manually done to not remove wanted files
Top3_df.loc[Top3_df['Channel']=='Red Royale']

,Channel,Video_Id,Published
104,Red Royale,O4ZYWRr7cso,11/12/2019


In [46]:
Top3_df.drop([104], inplace = True)

In [48]:
Top3_df.loc[Top3_df['Channel']=='Averroes']

,Channel,Video_Id,Published
116,Averroes,FhxlDVx5J3w,10/15/2019


In [49]:
Top3_df.drop([116], inplace = True)

In [50]:
Top3_df.loc[Top3_df['Channel']=='D3stri']

,Channel,Video_Id,Published
121,D3stri,2BcWkfK2e3s,10/26/2019
144,D3stri,q8wnIEpi9RY,10/31/2019


In [51]:
Top3_df.drop([121, 144], inplace = True)

In [52]:
Top3_df.loc[Top3_df['Channel']=='atack3000']

,Channel,Video_Id,Published
128,atack3000,a2zL3Wtvp3Y,11/03/2019


In [53]:
Top3_df.drop([128], inplace = True)

In [54]:
Top3_df.loc[Top3_df['Channel']=='Core Tops']

,Channel,Video_Id,Published
133,Core Tops,-SX-iiiCCw4,06/17/2019


In [55]:
Top3_df.drop([133], inplace = True)

In [58]:
Top3_df.loc[Top3_df['Channel']=='εlrubiusOMG']

,Channel,Video_Id,Published
39,εlrubiusOMG,rrdA_BiKvDw,11/04/2016


In [59]:
Top3_df.drop([39], inplace = True)

In [62]:
# creating a new csv file with dataframe cleaned.
Top3_df.to_csv('./data/Top3_cleaned.csv', sep='\t', index = False)

In [63]:
cleaned_video_id = list(Top3_df['Video_Id'])

In [66]:
# preparing the url to get the video details looping through the list of video ids.
url_a = 'https://www.googleapis.com/youtube/v3/videos?part=contentDetails&id='
url_b = f'&key={api_key}'

In [67]:
# looping to get the video details from each video Id.
videoDetails = []

for elem in cleaned_video_id:
    videoDetails.append(rq.get(url_a + elem + url_b).json())

In [69]:
# looping over videoDetails to get the duration of each video
durations = []

for i in range(len(videoDetails)):
    durations.append(videoDetails[i].get('items')[0].get('contentDetails').get('duration'))

In [74]:
Top3_df['Duration']=durations

In [78]:
Top3_df['Duration'] = Top3_df['Duration'].apply(lambda x: isodate.parse_duration(x).seconds)

In [80]:
Top3_df.reset_index(drop = True, inplace = True)

In [81]:
Top3_df

,Channel,Video_Id,Published,Duration
0,elrubiusOMG,NgGqA9Sy0tA,06/20/2019,722
1,elrubiusOMG,jUyxgUkNo9U,07/13/2019,602
2,elrubiusOMG,99jXthe_YYg,07/07/2019,655
3,elrubiusOMG,jmoDybW8j2k,06/06/2019,628
4,elrubiusOMG,geSbtIPEXg8,07/21/2019,983
...,...,...,...,...
138,Willyrex,jo20QQTjN90,08/01/2019,16321
139,Willyrex,0GywtMQ-5zM,07/20/2019,16292
140,Willyrex,hsa7mL4_wOY,10/27/2019,1360
141,Willyrex,T7u_w8x_tfE,11/02/2019,852


In [82]:
Top3_df.to_csv('./data/Top3_durations.csv', sep='\t', index = False)

In [85]:
# Importing rows to database table
from sqlalchemy import exc
num_rows = len(Top3_df)
#Iterate one row at a time
for i in range(num_rows):
    try:
       #Try inserting the row
       Top3_df.iloc[i:i+1].to_sql(name="video_table",con = engine,if_exists = 'append',index=False)
    except exc.IntegrityError:
       #Ignore duplicates
       pass